In [1]:
# importa os pacotes necessários
import numpy as np
import os, gc, cv2, random
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline 

import warnings
warnings.filterwarnings("ignore")

from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split
import sklearn.metrics
from sklearn.metrics import classification_report
from sklearn.model_selection import  cross_val_score

from sklearn import svm
from skimage.feature import greycomatrix, greycoprops
from skimage.feature import hog
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression

from sklearn.svm import LinearSVC
from sklearn.svm import SVC

In [2]:
%%time
df = pd.read_csv('kaggle/data_pca.csv')
df_labels = pd.read_csv('kaggle/data_labels.csv')
df.shape,df_labels.shape

CPU times: user 25.2 s, sys: 1.22 s, total: 26.5 s
Wall time: 40.3 s


In [3]:
X_train = df.iloc[:20556]
X_test  = df.iloc[20556:]
y_train = df_labels.iloc[:20556]
y_test  = df_labels.iloc[20556:]
X_train.shape,X_test.shape

((20556, 2500), (4444, 2500))

# Logistic Regression
Utilizando regressão logistica para ter uma base de comparação com os demais modelos.

In [4]:
%%time
logreg = LogisticRegression(solver='saga',n_jobs=4)
logreg.fit(X_train, y_train)

Wall time: 50.2 s


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=4, penalty='l2',
                   random_state=None, solver='saga', tol=0.0001, verbose=0,
                   warm_start=False)

In [5]:
# avalia predicao
y_pred = logreg.predict(X_test)
report = classification_report(y_test, y_pred,output_dict = True)
pd.DataFrame(report)

,cat,dog,accuracy,macro avg,weighted avg
f1-score,0.715465,0.710688,0.713096,0.713076,0.713076
precision,0.709606,0.716705,0.713096,0.713155,0.713155
recall,0.721422,0.704770,0.713096,0.713096,0.713096
support,2222.000000,2222.000000,0.713096,4444.000000,4444.000000


Somente os 1000 primeiros componentes.

In [6]:
%%time
logreg = LogisticRegression(solver='saga',n_jobs=4)
logreg.fit(X_train.iloc[:,:1000], y_train)

Wall time: 5.97 s


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=4, penalty='l2',
                   random_state=None, solver='saga', tol=0.0001, verbose=0,
                   warm_start=False)

In [7]:
# avalia predicao
y_pred = logreg.predict(X_test.iloc[:,:1000])
report = classification_report(y_test, y_pred,output_dict = True)
pd.DataFrame(report)

,cat,dog,accuracy,macro avg,weighted avg
f1-score,0.733875,0.735070,0.734473,0.734472,0.734472
precision,0.735533,0.733423,0.734473,0.734478,0.734478
recall,0.732223,0.736724,0.734473,0.734473,0.734473
support,2222.000000,2222.000000,0.734473,4444.000000,4444.000000


# Linear SVM
Classificador SVM otimizado para o kernel linear, pode não dar o mesmo 
resultado se comparado ao sklearn.svm.SVC(), mas tem um tempo de execução menor

In [8]:
%%time
svclassifier = LinearSVC()
svclassifier.fit(X_train, y_train)

Wall time: 2min 26s


LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
          intercept_scaling=1, loss='squared_hinge', max_iter=1000,
          multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
          verbose=0)

In [9]:
# avalia predicao
y_pred = svclassifier.predict(X_test)
report = classification_report(y_test, y_pred,output_dict = True)
pd.DataFrame(report)

,cat,dog,accuracy,macro avg,weighted avg
f1-score,0.673557,0.672526,0.673042,0.673041,0.673041
precision,0.672499,0.673589,0.673042,0.673044,0.673044
recall,0.674617,0.671467,0.673042,0.673042,0.673042
support,2222.000000,2222.000000,0.673042,4444.000000,4444.000000


Com os primeiros 1000 componentes.

In [6]:
%%time
svclassifier = LinearSVC()
svclassifier.fit(X_train.iloc[:,:1000], y_train)

CPU times: user 1min, sys: 9.66 ms, total: 1min
Wall time: 1min


In [7]:
# avalia predicao
y_pred = svclassifier.predict(X_test.iloc[:,:1000])
report = classification_report(y_test, y_pred,output_dict = True)
pd.DataFrame(report)

,cat,dog,accuracy,macro avg,weighted avg
f1-score,0.660150,0.666964,0.663591,0.663557,0.663557
precision,0.666973,0.660344,0.663591,0.663658,0.663658
recall,0.653465,0.673717,0.663591,0.663591,0.663591
support,2222.000000,2222.000000,0.663591,4444.000000,4444.000000


# Polinomial SVM
A função polinomial de 3° grau é a escolhida para o kernel

In [5]:
%%time
polysvclassifier = SVC(kernel='poly', tol=0.002)
polysvclassifier.fit(X_train, y_train)

Wall time: 27min 47s


SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
    kernel='poly', max_iter=-1, probability=False, random_state=None,
    shrinking=True, tol=0.002, verbose=False)

In [6]:
# avalia predicao
y_pred = polysvclassifier.predict(X_test)
report = classification_report(y_test, y_pred,output_dict = True)
pd.DataFrame(report)

,cat,dog,accuracy,macro avg,weighted avg
f1-score,0.612202,0.707832,0.666742,0.660017,0.660017
precision,0.731997,0.630137,0.666742,0.681067,0.681067
recall,0.526103,0.807381,0.666742,0.666742,0.666742
support,2222.000000,2222.000000,0.666742,4444.000000,4444.000000


Com os primeiros 1000 componentes.

In [14]:
%%time
polysvclassifier = SVC(kernel='poly', tol=0.002)
polysvclassifier.fit(X_train.iloc[:,:1000], y_train)

CPU times: user 32min 51s, sys: 1.31 s, total: 32min 53s
Wall time: 32min 55s


In [15]:
# avalia predicao
y_pred = polysvclassifier.predict(X_test.iloc[:,:1000])
report = classification_report(y_test, y_pred,output_dict = True)
pd.DataFrame(report)

,cat,dog,accuracy,macro avg,weighted avg
f1-score,0.724928,0.754894,0.740774,0.739911,0.739911
precision,0.772126,0.715900,0.740774,0.744013,0.744013
recall,0.683168,0.798380,0.740774,0.740774,0.740774
support,2222.000000,2222.000000,0.740774,4444.000000,4444.000000


Com os primeiros 1500 componentes.

In [4]:
%%time
polysvclassifier = SVC(kernel='poly', tol=0.002)
polysvclassifier.fit(X_train.iloc[:,:1500], y_train)

CPU times: user 30min 14s, sys: 451 ms, total: 30min 14s
Wall time: 30min 15s


In [5]:
# avalia predicao
y_pred = polysvclassifier.predict(X_test.iloc[:,:1500])
report = classification_report(y_test, y_pred,output_dict = True)
pd.DataFrame(report)

,cat,dog,accuracy,macro avg,weighted avg
f1-score,0.703286,0.737868,0.721647,0.720577,0.720577
precision,0.752953,0.697237,0.721647,0.725095,0.725095
recall,0.659766,0.783528,0.721647,0.721647,0.721647
support,2222.000000,2222.000000,0.721647,4444.000000,4444.000000


Com os primeiros 500 componentes.

In [6]:
%%time
polysvclassifier = SVC(kernel='poly', tol=0.002)
polysvclassifier.fit(X_train.iloc[:,:500], y_train)

CPU times: user 23min 28s, sys: 500 ms, total: 23min 28s
Wall time: 23min 29s


In [7]:
# avalia predicao
y_pred = polysvclassifier.predict(X_test.iloc[:,:500])
report = classification_report(y_test, y_pred,output_dict = True)
pd.DataFrame(report)

,cat,dog,accuracy,macro avg,weighted avg
f1-score,0.706422,0.743784,0.726373,0.725103,0.725103
precision,0.761979,0.699287,0.726373,0.730633,0.730633
recall,0.658416,0.794329,0.726373,0.726373,0.726373
support,2222.000000,2222.000000,0.726373,4444.000000,4444.000000
